In [1]:
model = "llama3.2"

#### Task 1: Simple Chain with Retrieval

**Objective:**

Implement a simple RAG chain with ChatOllama, HuggingFaceEmbeddings and Chroma. 

Process: 

1. Retrieve documents from chroma db based on query
2. Invoke chain with retrieved documents as input

**Task Description:**

- load llm model via ollama
- load huggingface embedding model (``model_name="sentence-transformers/all-mpnet-base-v2"``)
- create chroma db client
- create prompt template for summarization
- create simple chain with following steps: retrieved documents, prompt, model, output parser
- create query and perform similarity search with a query
- invoke chain and pass retrieved documents to the chain


**Useful links:**

- [RAG with Ollama](https://python.langchain.com/v0.2/docs/tutorials/local_rag/)


In [2]:
from langchain_ollama import ChatOllama

model = ChatOllama(model=model)

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model="sentence-transformers/all-mpnet-base-v2")

/Users/martin/Documents/DEV/build-your-own-chatbot/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/martin/Documents/DEV/build-your-own-chatbot/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
from langchain_chroma import Chroma
import chromadb
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

client = chromadb.HttpClient(
    host="localhost",
    port=8000,
    ssl=False,
    headers=None,
    settings=Settings(allow_reset=True, anonymized_telemetry=False),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

collection = client.get_or_create_collection("ai_model_book")

vector_db_from_client = Chroma(
    client=client,
    collection_name="ai_model_book",
    embedding_function=embedding_model,
)

In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Summarize the main themes in these retrieved docs: {docs}"
)


# Convert loaded documents into strings by concatenating their content
# and ignoring metadata
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = {"docs": format_docs} | prompt | model | StrOutputParser()

In [8]:
search_query = "Types of Machine Learning Systems"

docs = vector_db_from_client.similarity_search(search_query)

print(docs)

[Document(metadata={'page': 33, 'source': './AI_Book.pdf'}, page_content='Types of Machine Learning Systems\nThere are so many different types of Machine Learning systems that it is useful to\nclassify them in broad categories based on:\n•Whether or not they are trained with human supervision (supervised, unsuper‐\nvised, semisupervised, and Reinforcement Learning)\n•Whether or not they can learn incrementally on the fly (online versus batch\nlearning)\n•Whether they work by simply comparing new data points to known data points,\nor instead detect patterns in the training data and build a predictive model, much\nlike scientists do (instance-based versus model-based learning)\nThese criteria are not exclusive; you can combine them in any way you like. For\nexample, a state-of-the-art spam filter may learn on the fly using a deep neural net‐\nwork model trained using examples of spam and ham; this makes it an online, model-\nbased, supervised learning system.\nLet’s look at each of these

In [9]:
chain.invoke(docs)

'The main themes in these documents cover various types of Machine Learning (ML) systems. The document categorizes ML systems into four broad categories based on different criteria:\n\n1. Supervised/Unsupervised learning:\n- Supervised learning: Includes training with desired solutions or labels for supervised learning.\n- Unsupervised learning: No supervision during training.\n\n2. Batch and Online learning:\n- Batch learning: Requires all data to be processed before learning, typically offline.\n- Online learning: Can process incoming data incrementally without retraining.\n\n3. Instance-based versus model-based learning:\n- Instance-based learning models patterns in the training data rather than building a predictive model like supervised learning.\n- Model-based learning builds a predictive model from a set of examples.\n\n4. Reinforcement Learning (RL):\n- RL algorithms learn through trial and error, applying policies to find winning strategies for games or activities.\n\nThese ca

#### Task 2: Q&A with RAG

**Objective:**

Implement a Q/A retrieval chain with ChatOllama, HuggingFaceEmbeddings and Chroma

**Task Description:**

- create RAG-Q/A prompt template
- create retriever from vector db client (instead of manually passing in docs, we automatically retrieve them from our vector store based on the user question)
- create simple chain with following steps: retriever, formatting retrieved docs, user question, prompt, model, output parser
- create question for Q/A retrieval chain
- invoke chain and with question

**Useful links:**

- [RAG with Ollama](https://python.langchain.com/v0.2/docs/tutorials/local_rag/)

In [11]:
from langchain_core.runnables import RunnablePassthrough

prompt_template = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

<context>
{context}
</context>

Answer the following question:

{question}"""

rag_prompt = ChatPromptTemplate.from_template(prompt_template)

retriever = vector_db_from_client.as_retriever()

qa_rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | model
    | StrOutputParser()
)

In [15]:
qa_rag_chain

{
  context: VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0xffff1cd21210>)
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="\nYou are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\n\n<context>\n{context}\n</context>\n\nAnswer the following question:\n\n{question}"))])
| ChatOllama(model='sam4096/qwen2tools:1.5b', _client=<ollama._client.Client object at 0xffff65888190>, _async_client=<ollama._client.AsyncClient object at 0xffff6588c810>)
| StrOutputParser()

In [16]:
question = "What is supervised learning?"

qa_rag_chain.invoke(question)

"Supervised learning involves training an algorithm with labeled data, where the desired solutions are called labels. In this context, it's explained as a classifier or regressor that uses known examples for spam filtering."